# Full Waveform Inversion (FWI) Tutorial

This tutorial was prepared by Alexandre Olender olender@usp.br

This tutorial focuses on **Full Waveform Inversion (FWI)** using spyro's `FullWaveformInversion` class. Full Waveform Inversion is an advanced technique in seismic imaging that uses the full waveform information to reconstruct subsurface velocity models. Unlike traditional methods that use only arrival times or amplitudes, FWI leverages the complete seismic waveform, including phases, amplitudes, and frequencies.

**What is Full Waveform Inversion?**

Classical FWI is an iterative PDE-constrained optimization process that minimizes the difference between observed seismic data and synthetic data generated from a velocity model. The cost functional to be minimized varies depending on your desired FWI research. The process involves:

1. **Forward modeling**: Solving the wave equation to generate synthetic seismograms
2. **Residual calculation**: Computing the difference between observed and synthetic data  
3. **Adjoint modeling**: Computing gradients of the objective function related to the control variable (acoustic propagation velocity)
4. **Model update**: Updating the velocity model to reduce the data misfit

This tutorial demonstrates a complete FWI workflow, from generating synthetic "observed" data to performing the inversion and analyzing results.

**Prerequisites**: Basic understanding of seismic wave propagation and the forward modeling tutorial.

If you are running this notebook in Google Colab, please copy the following code into a code block before running the notebook:
```python
# For use in colab only:
try:
    import firedrake
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/firedrake-install-real.sh" -O "/tmp/firedrake-install.sh" && bash "/tmp/firedrake-install.sh"
    import firedrake

!pip install git+https://github.com/NDF-Poli-USP/spyro.git
```

## 1. Setup and Imports

We begin by importing the necessary libraries for FWI. In addition to Spyro, we'll need Firedrake for mesh operations, NumPy for numerical computations, and Matplotlib for visualization.

In [ ]:
# Enable inline plotting in the notebook
%matplotlib inline

# Import necessary libraries
import spyro
import firedrake as fire
import numpy as np
import matplotlib.pyplot as plt
import time
import warnings

# Suppress warnings for cleaner output
warnings.filterwarnings("ignore")

## 2. Define Problem Parameters

Let's set up our FWI problem parameters. We'll use a higher-order finite element method (degree 4) for better accuracy, and define our mesh, acquisition geometry, and time domain settings.

In [ ]:
# Define problem parameters
degree = 4
frequency = 5.0
final_time = 1.3

# Initialize the main dictionary for FWI parameters
dictionary = {}

# Finite element options - using triangular elements with lumped mass matrix
dictionary["options"] = {
    "cell_type": "T",  # Triangular elements (T) or quadrilaterals (Q)
    "variant": "lumped",  # lumped, equispaced or DG, default is lumped
    "degree": degree,  # Polynomial order (higher degree = better accuracy)
    "dimension": 2,  # 2D problem
}

# Parallelism settings
dictionary["parallelism"] = {
    "type": "automatic",  # Automatic core distribution
}

In [ ]:
# Mesh parameters - define the computational domain
dictionary["mesh"] = {
    "length_z": 2.0,  # Depth in km (always positive)
    "length_x": 2.0,  # Width in km (always positive)  
    "length_y": 0.0,  # Thickness in km (0 for 2D)
}

# Acquisition geometry - sources and receivers
dictionary["acquisition"] = {
    "source_type": "ricker",  # Ricker wavelet source
    "source_locations": spyro.create_transect((-0.35, 0.5), (-0.35, 1.5), 1),  # Single source
    "frequency": frequency,  # Dominant frequency in Hz
    "delay": 1.0/frequency,  # Source delay (1 period)
    "delay_type": "time",
    "receiver_locations": spyro.create_transect((-1.65, 0.5), (-1.65, 1.5), 200),  # 200 receivers
}

print(f"Source location: {dictionary['acquisition']['source_locations']}")
print(f"Number of receivers: {len(dictionary['acquisition']['receiver_locations'])}")
print(f"Receiver spacing: {(1.0) / 199:.4f} km")

In [ ]:
# Absorbing boundary conditions - prevent reflections from domain boundaries
dictionary["absorving_boundary_conditions"] = {
    "status": True,
    "damping_type": "local",  # Local damping near boundaries
}

# Time domain parameters
dictionary["time_axis"] = {
    "initial_time": 0.0,  # Start time
    "final_time": final_time,  # End time (must be long enough for waves to propagate)
    "dt": 0.0001,  # Time step (small for stability with high frequency)
    "amplitude": 1,  # Source amplitude
    "output_frequency": 100,  # Save solution every 100 time steps for visualization
    "gradient_sampling_frequency": 1,  # Save every time step for gradient computation
}

# Visualization and output settings  
dictionary["visualization"] = {
    "forward_output": True,
    "forward_output_filename": "results/forward_output.pvd",
    "fwi_velocity_model_output": False,
    "velocity_model_filename": None,
    "gradient_output": False,
    "gradient_filename": "results/Gradient.pvd",
    "adjoint_output": False,
    "adjoint_filename": None,
    "debug_output": False,
}

## 3. Create Real Data (Synthetic Example)

In real-world FWI, we would have observed seismic data from field measurements. For this tutorial, we'll generate synthetic "real" data by solving the forward problem with a known complex velocity model. This synthetic data will serve as our target for the inversion.

First, let's define a utility function to calculate the optimal cells per wavelength based on the polynomial degree.

In [ ]:
def cells_per_wavelength(degree):
    """
    Calculate optimal cells per wavelength for different polynomial degrees.
    This ensures adequate resolution for wave propagation.
    """
    cell_per_wavelength_dictionary = {
        'kmv2tri': 7.20,
        'kmv3tri': 3.97,
        'kmv4tri': 2.67,  # For degree 4 triangular elements
        'kmv5tri': 2.03,
        'kmv6tri': 1.5,
        'kmv2tet': 6.12,
        'kmv3tet': 3.72,
    }
    
    cell_type = 'tri'  # Using triangular elements
    key = 'kmv' + str(degree) + cell_type
    
    return cell_per_wavelength_dictionary.get(key)

# Calculate cells per wavelength for our degree
cpw = cells_per_wavelength(degree)
print(f"Using {cpw} cells per wavelength for degree {degree} triangular elements")

## 4. Set Up Real Mesh and Velocity Model

Now we'll create the "true" velocity model that will generate our synthetic observed data. This model contains both circular and rectangular anomalies to create realistic complexity for the inversion challenge.

In [ ]:
def generate_real_data():
    """
    Generate synthetic 'observed' data using a complex velocity model.
    This represents what we would measure in the field.
    """
    
    # Create FWI object for generating real data
    fwi = spyro.FullWaveformInversion(dictionary=dictionary)
    
    # Set up mesh with fine resolution for accurate forward modeling
    fwi.set_real_mesh(input_mesh_parameters={
        "edge_length": 0.05,  # Fine mesh for accurate modeling
        "mesh_type": "firedrake_mesh"
    })
    
    # Define the true velocity model with anomalies
    # Get mesh coordinates
    mesh_z = fwi.mesh_z  # Depth coordinate
    mesh_x = fwi.mesh_x  # Horizontal coordinate
    
    # Define circular anomaly parameters
    center_z = -1.0  # Center depth (negative because z points down)
    center_x = 1.0   # Center x-position  
    radius = 0.4     # Radius of circular anomaly
    
    # Define rectangular anomaly parameters
    square_top_z = -0.8
    square_bot_z = -1.2  
    square_left_x = 0.8
    square_right_x = 1.2
    
    # Create velocity model using Firedrake conditionals
    # Background velocity: 2.5 km/s
    # Circular anomaly: 3.0 km/s (higher velocity)
    # Rectangular anomaly: 3.5 km/s (highest velocity)
    
    # First create circular anomaly
    cond = fire.conditional(
        (mesh_z - center_z)**2 + (mesh_x - center_x)**2 < radius**2, 
        3.0,  # High velocity inside circle
        2.5   # Background velocity
    )
    
    # Add rectangular anomaly (overwrites circular where they overlap)
    cond = fire.conditional(
        fire.And(
            fire.And(mesh_z < square_top_z, mesh_z > square_bot_z),
            fire.And(mesh_x > square_left_x, mesh_x < square_right_x)
        ),
        3.5,  # Very high velocity in rectangle
        cond  # Previous conditional (background + circle)
    )
    
    return fwi, cond

# Generate the real data setup
print("Setting up real velocity model...")
fwi_real, velocity_model = generate_real_data()

## 5. Generate Real Shot Records

Now we'll use the true velocity model to generate synthetic seismograms. These will serve as our "observed" data for the inversion. In a real application, this would be replaced by field measurements.

In [ ]:
# Set the true velocity model
fwi_real.set_real_velocity_model(
    conditional=velocity_model, 
    output=True,  # Enable output for visualization
    dg_velocity_model=False  # Use CG space instead of DG
)

# Generate synthetic shot records (this may take a few minutes)
print("Generating synthetic shot records...")
print("This simulates what we would measure in the field...")

shot_filename = f"shots/shot_record_f{frequency}_"

fwi_real.generate_real_shot_record(
    plot_model=True,  # Plot the true velocity model
    save_shot_record=True,  # Save shot records to disk
    shot_filename=shot_filename,  # Filename prefix
    high_resolution_model=True  # Use high resolution for accurate modeling
)

print(f"Synthetic data saved with prefix: {shot_filename}")
print("The true velocity model has been plotted for reference.")

## 6. Initialize FWI Object

Now we'll set up the FWI object for the actual inversion process. We need to tell it where to find the synthetic "observed" data we just generated.

In [ ]:
# Configure the inversion dictionary to point to our synthetic data
dictionary["inversion"] = {
    "perform_fwi": True,
    "real_shot_record_file": shot_filename,  # Path to our synthetic "observed" data
}

# Create new FWI object for the inversion
print("Initializing FWI object for inversion...")
fwi = spyro.FullWaveformInversion(dictionary=dictionary)
print("FWI object created successfully!")

## 7. Set Up Guess Mesh and Initial Model

For FWI, we need an initial guess for the velocity model. In real applications, this might come from other geophysical methods or geological knowledge. Here, we'll start with a simple constant velocity model.

In [ ]:
# First, create a simple mesh to generate a velocity grid
print("Setting up initial guess mesh and model...")

fwi.set_guess_mesh(input_mesh_parameters={
    "mesh_type": "firedrake_mesh", 
    "edge_length": 0.05
})

# Start with a homogeneous velocity model (our initial guess)
initial_velocity = 1.5  # km/s - much lower than the true model
fwi.set_guess_velocity_model(constant=initial_velocity)

print(f"Initial guess velocity: {initial_velocity} km/s")
print("(This is intentionally different from the true model to test the inversion)")

# Convert velocity model to grid format for spyro_mesh
print("Converting velocity to grid format...")
grid_data = spyro.utils.velocity_to_grid(fwi, 0.01, output=True)
print("Grid conversion completed.")

## 8. Configure Inversion Parameters

Now we'll set up the mesh and gradient masking for the inversion. The gradient mask defines the region where we allow the velocity to be updated during the inversion.

In [ ]:
# Define gradient mask boundaries - region where inversion is allowed
# This prevents updating velocities near boundaries where resolution is poor
mask_boundaries = {
    "z_min": -1.55,  # Minimum depth to invert
    "z_max": -0.45,  # Maximum depth to invert  
    "x_min": 0.45,   # Minimum x position to invert
    "x_max": 1.55,   # Maximum x position to invert
}

print("Gradient mask boundaries:")
for key, value in mask_boundaries.items():
    print(f"  {key}: {value}")

# Set up the final mesh for inversion using spyro_mesh
# This creates a mesh adapted to the wavelength requirements
fwi.set_guess_mesh(input_mesh_parameters={
    "mesh_type": "spyro_mesh",
    "cells_per_wavelength": 2.7,  # Coarser than forward modeling for efficiency
    "grid_velocity_data": grid_data,
    "gradient_mask": mask_boundaries,
    "output_filename": "test.vtk"  # Save mesh for inspection
})

# Set initial velocity model for inversion (different from true model)
inversion_initial_velocity = 2.5  # km/s
fwi.set_guess_velocity_model(constant=inversion_initial_velocity)

print(f"Inversion starting velocity: {inversion_initial_velocity} km/s")
print("Mesh setup complete and ready for inversion!")

## 9. Run Full Waveform Inversion

Now comes the main event - running the FWI algorithm! This is an iterative optimization process that will:

1. **Forward modeling**: Compute synthetic data using current velocity model
2. **Residual calculation**: Compare synthetic and observed data
3. **Adjoint modeling**: Compute gradients of the objective function
4. **Model update**: Update the velocity model to reduce data misfit
5. **Repeat** until convergence or maximum iterations

**Note**: This process can take several minutes to complete depending on the number of iterations and computational resources.

In [ ]:
# Define FWI parameters
vmin = 2.5  # Minimum allowed velocity (km/s)
vmax = 3.5  # Maximum allowed velocity (km/s)
maxiter = 30  # Maximum number of iterations

print("Starting Full Waveform Inversion...")
print(f"Velocity bounds: [{vmin}, {vmax}] km/s")
print(f"Maximum iterations: {maxiter}")
print(f"Target: Recover the true model from initial guess of {inversion_initial_velocity} km/s")
print()
print("FWI Progress:")
print("=" * 50)

# Record start time
t_start = time.time()

# Run the FWI algorithm
fwi.run_fwi(vmin=vmin, vmax=vmax, maxiter=maxiter)

# Record end time
t_end = time.time()
total_time = t_end - t_start

print("=" * 50)
print(f"FWI completed in {total_time:.2f} seconds!")
print("Inversion finished successfully.")

## 10. Visualize Results

Let's examine the results of our FWI! We'll visualize the inverted velocity model and analyze how well it recovered the true model structure.

In [ ]:
# Plot the final inverted velocity model
print("Plotting inverted velocity model...")

try:
    # Plot the inverted model using Spyro's plotting utilities
    spyro.plots.plot_model(fwi, 
                          filename="inverted_model.png", 
                          flip_axis=False, 
                          show=True)
    print("Inverted velocity model plotted successfully!")
    
except Exception as e:
    print(f"Error plotting model: {e}")
    print("You can visualize the results using ParaView with the generated .pvd files")

# Access the final velocity model
final_velocity = fwi.vp
print(f"Final velocity model type: {type(final_velocity)}")

# Get velocity statistics
if hasattr(final_velocity, 'dat'):
    velocity_data = final_velocity.dat.data[:]
    print(f"Velocity statistics:")
    print(f"  Minimum: {np.min(velocity_data):.3f} km/s")
    print(f"  Maximum: {np.max(velocity_data):.3f} km/s")
    print(f"  Mean: {np.mean(velocity_data):.3f} km/s")
    print(f"  Standard deviation: {np.std(velocity_data):.3f} km/s")

In [ ]:
# Summary of the FWI experiment
print("\n" + "="*60)
print("FULL WAVEFORM INVERSION SUMMARY")
print("="*60)
print(f"📊 Problem Setup:")
print(f"   • Domain size: {dictionary['mesh']['length_x']} × {dictionary['mesh']['length_z']} km")
print(f"   • Frequency: {frequency} Hz")
print(f"   • Polynomial degree: {degree}")
print(f"   • Number of receivers: {len(dictionary['acquisition']['receiver_locations'])}")

print(f"\n🎯 True Model:")
print(f"   • Background velocity: 2.5 km/s")
print(f"   • Circular anomaly: 3.0 km/s (radius {radius} km)")
print(f"   • Rectangular anomaly: 3.5 km/s")

print(f"\n🔄 Inversion Setup:")
print(f"   • Initial guess: {inversion_initial_velocity} km/s (constant)")
print(f"   • Velocity bounds: [{vmin}, {vmax}] km/s")
print(f"   • Maximum iterations: {maxiter}")
print(f"   • Total runtime: {total_time:.2f} seconds")

print(f"\n✅ Results:")
if 'velocity_data' in locals():
    recovered_range = np.max(velocity_data) - np.min(velocity_data)
    true_range = 3.5 - 2.5
    print(f"   • Recovered velocity range: {recovered_range:.3f} km/s")
    print(f"   • True velocity range: {true_range:.3f} km/s")
    print(f"   • Range recovery: {(recovered_range/true_range)*100:.1f}%")

print(f"\n📁 Output Files:")
print(f"   • Synthetic data: {shot_filename}*")
print(f"   • Mesh file: test.vtk")
print(f"   • Model plots: inverted_model.png")
print("="*60)

## Conclusion and Next Steps

Congratulations! You have successfully completed a Full Waveform Inversion tutorial using Spyro. Here's what we accomplished:

### What We Did:
1. **Generated synthetic data** using a complex velocity model with circular and rectangular anomalies
2. **Set up an FWI problem** with proper mesh, acquisition geometry, and time domain parameters
3. **Performed iterative inversion** starting from a simple constant velocity model
4. **Recovered subsurface structure** that was initially unknown to the algorithm

### Key FWI Concepts Demonstrated:
- **Forward modeling**: Computing synthetic seismograms from velocity models
- **Data misfit**: Measuring differences between observed and synthetic data
- **Gradient computation**: Using adjoint methods to compute model updates
- **Iterative optimization**: Gradually improving the velocity model

### Real-World Applications:
- **Oil and gas exploration**: Imaging complex subsurface structures
- **Earthquake seismology**: Understanding Earth's interior structure  
- **Geothermal exploration**: Mapping temperature and fluid distributions
- **Carbon sequestration**: Monitoring CO₂ injection and storage

### Further Exploration:
- Try different initial models and see how it affects convergence
- Experiment with different frequencies and acquisition geometries
- Explore multi-scale FWI by starting with low frequencies and gradually increasing
- Investigate 3D FWI problems for more realistic scenarios

### Advanced Topics:
- **Multi-parameter inversion**: Simultaneously inverting for velocity and density
- **Elastic FWI**: Including S-wave velocities and more complex physics
- **Time-lapse FWI**: Monitoring changes over time
- **Machine learning integration**: Using AI to improve FWI workflows

For more advanced tutorials and examples, refer to the Spyro documentation and additional notebook tutorials in this repository.